<center><span style="font-size: 24px;font-weight: bold">SDXL Models - Dreamscape Notebook 1.9</span></center>
<center>Follow tutorial on my  <a href="www.youtube.com/@dreamscapeyoutube">Youtube Channel</a>. Join our <a href="https://discord.gg/VCqcfZH2jV">Discord Community</a> for support.</center>

<h4><b>🧼 Step 1. Fresh Clean Environments </b>(launch if it's your first start)</h4>
If you have a healthy and working install, SKIP to STEP 3. This can be launched in CPU mode.

In [ ]:
!rm -r /home/studio-lab-user/.cache
!rm -r /home/studio-lab-user/.conda
!conda remove -n "env_name" --all
!rm -rf /home/studio-lab-user/*

☢️*Your Browser Panel (on the left) should be empty.*<br>
☢️You need to restart the Jupyter Lab now (before running next cell).

In [ ]:
import subprocess

# Run the command and capture the output
result = subprocess.run(['du', '-h', '--max-depth=1', '/home/studio-lab-user'], capture_output=True, text=True)

# Check if the desired line is present in the output
if '1.1G\t/home/studio-lab-user/.conda' in result.stdout:
    print("Ok, your conda is clean (1.1GB) please continue with Step 2")

<h4><b>🔥 Step 2. Install essentials </b>(launch if it's your first start)</h4>
This will install essentials. This can be launched in CPU mode. Be patient it might take a while.

In [ ]:
#Install all essentials for stable diffusion
!conda install -c conda-forge libglib -y & conda update -n base conda -y
%conda install -q -y aria2
%conda install -q -y libglib2.0-0
%conda install -q -y glib
%cd /home/studio-lab-user/
!git clone --depth 1 https://github.com/dreamscapeai/stable-diffusion-webui
%cd /home/studio-lab-user/stable-diffusion-webui/embeddings
!git clone --depth 1 https://github.com/dreamscapeai/negativekit
%cd /home/studio-lab-user/stable-diffusion-webui/extensions
!git clone --depth 1 https://github.com/ahgsql/StyleSelectorXL
!git clone --depth 1 https://github.com/etherealxx/batchlinks-webui
!git clone --depth 1 https://github.com/zanllp/sd-webui-infinite-image-browsing
!git clone --depth 1 https://github.com/dreamscapeai/ultimatesdupscale
!git clone --depth 1 https://github.com/dreamscapeai/tagcomplete
!git clone --depth 1 https://github.com/IDEA-Research/DWPose

<h4><b>💾 Step 3. Downloads Models and Checkpoints </b></h4>
The model are downloaded in tmp folder, which will be emptied at every new runtime. This MUST be launched in GPU MODE.

In [ ]:
!rm -rf /home/studio-lab-user/tmp
!rm -rf /home/studio-lab-user/stable-diffusion-webui/models/Stable-diffusion/tmp_models
!ln -vs /tmp ~/tmp
!ln -vs /tmp/models /home/studio-lab-user/stable-diffusion-webui/models/Stable-diffusion/tmp_models
!mkdir -p /tmp/models
%cd ~/stable-diffusion-webui/models/Stable-diffusion/tmp_models
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd_xl_base_1.0/resolve/main/sd_xl_base_1.0_0.9vae.safetensors   -o sdxl_base.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd_xl_refiner_1.0/resolve/main/sd_xl_refiner_1.0_0.9vae.safetensors   -o sdxl_refiner.safetensors
%cd ~/stable-diffusion-webui/models/VAE
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sdxl_vae/resolve/main/sdxl_vae.safetensors -o sdxl_vae.vae.safetensors

<h4><b>🌟 Step 4. BOOT LAUNCHER WEBUI (MUST BE in GPU Mode)</b></h4> 
Don't forget to paste your NGROK token below. Create one token here https://dashboard.ngrok.com/get-started/your-authtoken

In [ ]:
#Don't forget to paste your NGROK token here
%cd ~/stable-diffusion-webui
!python launch.py --ngrok PASTE_YOUR_TOKEN_HERE --listen --xformers --no-half-vae --enable-insecure-extension-access --theme dark --gradio-queue

In [ ]:
#Check your available storage
!df -h | grep -E 'Avail|home'

<h4><b>🖼️ Step 5. Download all your outputs</b></h4>
If your setup folder doesn't have problem, skip this step.

In [ ]:
#download and compress all outputs
import os
import zipfile
def zip_folder(input_folder, output_folder, max_size_mb=20):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    zip_number = 1
    current_zip_size = 0
    current_zip_name = os.path.join(output_folder, f"part_{zip_number}.zip")    
    with zipfile.ZipFile(current_zip_name, 'w', zipfile.ZIP_DEFLATED) as current_zip:
        for root, _, files in os.walk(input_folder):
            for file in files:
                file_path = os.path.join(root, file)
                file_size = os.path.getsize(file_path)           
                # Check if adding the current file exceeds the max size
                if current_zip_size + file_size > max_size_mb * 1024 * 1024:
                    # Close the current zip file
                    current_zip.close()             
                    # Start a new zip file
                    zip_number += 1
                    current_zip_name = os.path.join(output_folder, f"part_{zip_number}.zip")
                    current_zip = zipfile.ZipFile(current_zip_name, 'w', zipfile.ZIP_DEFLATED)
                    current_zip_size = 0  
                # Add the file to the current zip
                current_zip.write(file_path, os.path.relpath(file_path, input_folder))
                current_zip_size += file_size
# Specify the input folder to be zipped
input_folder = 'stable-diffusion-webui/outputs/txt2img-images'  # Replace with the path to your input folder
# Specify the output folder where zip files will be saved
output_folder = 'outputs'  # Replace with the path to your output folder
# Specify the maximum size for each zip file in MB
max_size_mb = 200
# Create zip files with a maximum size of 20MB
zip_folder(input_folder, output_folder, max_size_mb)

In [ ]:
#delete and empty the OUTPUT folder
!rm -rf /home/studio-lab-user/stable-diffusion-webui/outputs/*

<h4><b> Backup Tools </b></h4>
If your setup folder doesn't have problem, skip this step.

In [ ]:
!curl -Lo "/home/studio-lab-user/stable-diffusion-webui/models/VAE/sdxl_vae.safetensors" "https://huggingface.co/ckpt/sdxl_vae/resolve/main/sdxl_vae.safetensors"

In [ ]:
!curl -Lo "/home/studio-lab-user/stable-diffusion-webui/models/Stable-diffusion/epicphotogasm.safetensors" "https://civitai.com/api/download/models/145885"
!curl -Lo "/home/studio-lab-user/stable-diffusion-webui/models/Stable-diffusion/RealvisXL.safetensors" "https://civitai.com/api/download/models/154590"

In [ ]:
# tmp
!ln -vs /tmp/controlnet_models /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models
!mkdir -p /tmp/controlnet_models
%cd ~/stable-diffusion-webui/extensions/sd-webui-controlnet/models

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors             -o control_v11e_sd15_ip2p_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors          -o control_v11e_sd15_shuffle_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny_fp16.safetensors            -o control_v11p_sd15_canny_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors          -o control_v11f1p_sd15_depth_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors          -o control_v11p_sd15_inpaint_fp16.safetensors
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart_fp16.safetensors          -o control_v11p_sd15_lineart_fp16.safetensors
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors             -o control_v11p_sd15_mlsd_fp16.safetensors
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors        -o control_v11p_sd15_normalbae_fp16.safetensors
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose_fp16.safetensors         -o control_v11p_sd15_openpose_fp16.safetensors
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble_fp16.safetensors         -o control_v11p_sd15_scribble_fp16.safetensors
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg_fp16.safetensors              -o control_v11p_sd15_seg_fp16.safetensors
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge_fp16.safetensors         -o control_v11p_sd15_softedge_fp16.safetensors
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors  -o control_v11p_sd15s2_lineart_anime_fp16.safetensors
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors           -o control_v11f1e_sd15_tile_fp16.safetensors



#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11e_sd15_ip2p_fp16.yaml                        -o control_v11e_sd15_ip2p_fp16.yaml
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11e_sd15_shuffle_fp16.yaml                     -o control_v11e_sd15_shuffle_fp16.yaml
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_canny_fp16.yaml                       -o control_v11p_sd15_canny_fp16.yaml
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1p_sd15_depth_fp16.yaml                     -o control_v11f1p_sd15_depth_fp16.yaml
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_inpaint_fp16.yaml                     -o control_v11p_sd15_inpaint_fp16.yaml
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_lineart_fp16.yaml                     -o control_v11p_sd15_lineart_fp16.yaml
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_mlsd_fp16.yaml                        -o control_v11p_sd15_mlsd_fp16.yaml
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_normalbae_fp16.yaml                   -o control_v11p_sd15_normalbae_fp16.yaml
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_openpose_fp16.yaml                    -o control_v11p_sd15_openpose_fp16.yaml
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_scribble_fp16.yaml                    -o control_v11p_sd15_scribble_fp16.yaml
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_seg_fp16.yaml                         -o control_v11p_sd15_seg_fp16.yaml
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_softedge_fp16.yaml                    -o control_v11p_sd15_softedge_fp16.yaml
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15s2_lineart_anime_fp16.yaml             -o control_v11p_sd15s2_lineart_anime_fp16.yaml
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1e_sd15_tile_fp16.yaml                      -o control_v11f1e_sd15_tile_fp16.yaml

In [ ]:
#List all folders and sizes
!du -h --max-depth=1 /home/studio-lab-user

In [ ]:
# create symlink to temporary folder
!ln -vs /tmp ~/tmp
#
!ln -vs /tmp/outputs /home/studio-lab-user/stable-diffusion-webui/outputs
#
!ln -vs /tmp/models /home/studio-lab-user/stable-diffusion-webui/models/Stable-diffusion/tmp_models
#
!mkdir ~/stable-diffusion-webui/models/Lora
!ln -vs /tmp/Lora /home/studio-lab-user/stable-diffusion-webui/models/Lora/tmp_Lora

In [ ]:
!conda list

In [ ]:
#clean remove all env, cache and file
!rm -r /home/studio-lab-user/.cache
!rm -r /home/studio-lab-user/.stable-diffusion-webui
!conda remove -n "env_name" --all
!rm -rf /home/studio-lab-user/*

In [ ]:
!rm -rf /home/studio-lab-user/tmp
!rm -rf /home/studio-lab-user/stable-diffusion-webui/models/Stable-diffusion/tmp_models
!rm -rf /home/studio-lab-user/stable-diffusion-webui/


#delete and empty the MODELS folder
!rm -rf /home/studio-lab-user/stable-diffusion-webui/models/Stable-diffusion/*

# persistent
#%cd ~/asd/models/Stable-diffusion
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/77744 -o Meichidark_Mix_V2.safetensors

In [ ]:
!curl -Lo "/home/studio-lab-user/stable-diffusion-webui/models/Stable-diffusion/sdxl_base.safetensors" "https://huggingface.co/ckpt/sd_xl_base_1.0/resolve/main/sd_xl_base_1.0_0.9vae.safetensors"
!curl -Lo "/home/studio-lab-user/stable-diffusion-webui/models/Stable-diffusion/sdxl_refiner.safetensors" "https://huggingface.co/ckpt/sd_xl_refiner_1.0/resolve/main/sd_xl_refiner_1.0_0.9vae.safetensors"


BELOW ONLY FOR TROUBLESHOOTING

In [ ]:
%cd ~/stable-diffusion-webui
!pip install -r requirements.txt

In [ ]:
%cd ~/stable-diffusion-webui
!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --ngrok auth

In [ ]:
import torch
torch.cuda.get_device_name(0)

In [ ]:
import torch
print(torch.version.cuda)

In [ ]:
!pip uninstall torchvision -y

In [ ]:
!pip install torchvision==0.10.0

In [ ]:
!pip install torch==1.10.0

In [ ]:
!pip install xformers==0.0.20 tomesd==0.1.3 pytorch-lightning==1.9.4 accelerate==0.21.0

In [ ]:
!conda install -c conda-forge libglib -y